# Exploring Hacker News Posts

## Introduction

Hacker News is a site started by the startup incubator Y Combinator, where user-submitted stories (known as "posts") are voted and commented upon, similar to reddit. Hacker News is extremely popular in technology and startup circles, and posts that make it to the top of Hacker News' listings can get hundreds of thousands of visitors as a result.

I was specifically interested in posts whose titles begin with either Ask HN or Show HN. Users submit Ask HN posts to ask the Hacker News community a specific question. Below are a couple examples:

We'll compare these two types of posts to determine the following:

- Do Ask HN or Show HN receive more comments on average?
- Do posts created at a certain time receive more comments on average?

In [1]:
# Read the `hacker_news.csv` file

from csv import reader
opened_file = open("hacker_news.csv")   
read_file = reader(opened_file)
data = list(read_file)
hn = data

hn[:5]

[['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at'],
 ['12224879',
  'Interactive Dynamic Video',
  'http://www.interactivedynamicvideo.com/',
  '386',
  '52',
  'ne0phyte',
  '8/4/2016 11:52'],
 ['10975351',
  'How to Use Open Source and Shut the Fuck Up at the Same Time',
  'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/',
  '39',
  '10',
  'josep2',
  '1/26/2016 19:30'],
 ['11964716',
  "Florida DJs May Face Felony for April Fools' Water Joke",
  'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/',
  '2',
  '1',
  'vezycash',
  '6/23/2016 22:20'],
 ['11919867',
  'Technology ventures: From Idea to Enterprise',
  'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429',
  '3',
  '1',
  'hswarna',
  '6/17/2016 0:01']]

## Removing Headers from a List of Lists

In [2]:
# Removing Headers

headers = data[0]
hn = hn[1:]

print(headers)
print(hn[:5])

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']
[['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52'], ['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30'], ['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20'], ['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01'], ['10301696', 'Note by Note: The Making of Steinway L1037 (2007)', 'http://www.nytimes.com/2007/11/07/movies/07stein.html?_r=0', '8', '2', 'walterbell', '9/30/2015 4:12']]


## Extracting Ask HN and Show HN Posts

First, we'll identify posts that begin with either Ask HN or Show HN and separate the data for those two types of posts into different lists. Separating the data makes it easier to analyze in the following steps.

In [3]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    if title.lower().startswith("ask hn"):
        ask_posts.append(row)
    elif title.lower().startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

print(ask_posts[:5])

1744
1162
17194
[['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55'], ['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43'], ['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14'], ['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20'], ['10394168', 'Ask HN: Someone offered to buy my browser extension from me. What now?', '', '28', '17', 'roykolak', '10/15/2015 16:38']]


## Calculating the Average Number of Comments for Ask HN and Show HN Posts

Now that we separated ask posts and show posts into different lists, we'll calculate the average number of comments each type of post receives.

In [4]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = int(row[4])
    total_ask_comments += num_comments

avg_ask_comments = round(total_ask_comments/len(ask_posts))

print("Ask Posts:",avg_ask_comments)

total_show_comments = 0

for row in show_posts:
    num_comments = int(row[4])
    total_show_comments += num_comments

avg_show_comments = round(total_show_comments/len(show_posts))

print("Show Posts:",avg_show_comments)
    
    

Ask Posts: 14
Show Posts: 10


On average, ask posts in our sample receive approximately 14 comments, whereas show posts receive approximately 10. Since ask posts are more likely to receive comments, we'll focus our remaining analysis just on these posts.

## Finding the Amount of Ask Posts and Comments by Hour Created

Next, we'll determine if we can maximize the amount of comments an ask post receives by creating it at a certain time. First, we'll find the amount of ask posts created during each hour of day, along with the number of comments those posts received. Then, we'll calculate the average amount of comments ask posts created at each hour of the day receive.

In [5]:
#import datetime modoule

import datetime as dt

result_list = []

for row in ask_posts: 
    result_list.append([row[6],int(row[4])])
       
counts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    created_at = row[0]
    date_format = "%m/%d/%Y %H:%M"
    created_at_obj = dt.datetime.strptime(created_at, date_format)
    hour = created_at_obj.strftime("%H")
    if hour not in counts_by_hour:
        counts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        counts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]

print(comments_by_hour)

{'23': 543, '17': 1146, '00': 447, '05': 464, '06': 397, '11': 641, '19': 1188, '02': 1381, '18': 1439, '07': 267, '13': 1253, '04': 337, '09': 251, '22': 479, '10': 793, '20': 1722, '03': 421, '15': 4477, '01': 683, '12': 687, '08': 492, '14': 1416, '21': 1745, '16': 1814}


## Calculating the Average Number of Comments for Ask HN Posts by Hour

In [6]:
# Calculate the average amount of comments `Ask HN` posts created at each hour of the day receive.
avg_by_hour = []

for hr in comments_by_hour:
    avg_by_hour.append([hr, comments_by_hour[hr]/counts_by_hour[hr]])
    
avg_by_hour

[['23', 7.985294117647059],
 ['17', 11.46],
 ['00', 8.127272727272727],
 ['05', 10.08695652173913],
 ['06', 9.022727272727273],
 ['11', 11.051724137931034],
 ['19', 10.8],
 ['02', 23.810344827586206],
 ['18', 13.20183486238532],
 ['07', 7.852941176470588],
 ['13', 14.741176470588234],
 ['04', 7.170212765957447],
 ['09', 5.5777777777777775],
 ['22', 6.746478873239437],
 ['10', 13.440677966101696],
 ['20', 21.525],
 ['03', 7.796296296296297],
 ['15', 38.5948275862069],
 ['01', 11.383333333333333],
 ['12', 9.41095890410959],
 ['08', 10.25],
 ['14', 13.233644859813085],
 ['21', 16.009174311926607],
 ['16', 16.796296296296298]]

## Sorting and Printing Values from a List of Lists

In [16]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])

print(swap_avg_by_hour)

sorted_swap = sorted(swap_avg_by_hour , reverse = True)

    


[[7.985294117647059, '23'], [11.46, '17'], [8.127272727272727, '00'], [10.08695652173913, '05'], [9.022727272727273, '06'], [11.051724137931034, '11'], [10.8, '19'], [23.810344827586206, '02'], [13.20183486238532, '18'], [7.852941176470588, '07'], [14.741176470588234, '13'], [7.170212765957447, '04'], [5.5777777777777775, '09'], [6.746478873239437, '22'], [13.440677966101696, '10'], [21.525, '20'], [7.796296296296297, '03'], [38.5948275862069, '15'], [11.383333333333333, '01'], [9.41095890410959, '12'], [10.25, '08'], [13.233644859813085, '14'], [16.009174311926607, '21'], [16.796296296296298, '16']]


In [19]:
# Sort the values and print the the 5 hours with the highest average comments.

print("\n Top 5 Hours for Ask Posts Comments")

for row in sorted_swap[:5]:
   print("{0}:00: {1:.2f} average comments per post.".format(row[1], row[0]))

#Alternative solution using datetime.strptime & strftime()
print("\n Top 5 Hours for 'Ask HN' Comments")
for avg, hr in sorted_swap[:5]:
    print(
        "{}: {:.2f} average comments per post.".format(
            dt.datetime.strptime(hr, "%H").strftime("%H:%M"),avg
        )
    )


 Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post.
02:00: 23.81 average comments per post.
20:00: 21.52 average comments per post.
16:00: 16.80 average comments per post.
21:00: 16.01 average comments per post.

 Top 5 Hours for 'Ask HN' Comments
15:00: 38.59 average comments per post.
02:00: 23.81 average comments per post.
20:00: 21.52 average comments per post.
16:00: 16.80 average comments per post.
21:00: 16.01 average comments per post.


The hour that receives the most comments per post on average is 15:00, with an average of 38.59 comments per post. There's about a 60% increase in the number of comments between the hours with the highest and second highest average number of comments.

According to the data set [documentation](https://www.kaggle.com/hacker-news/hacker-news-posts/home), the timezone used is Eastern Time in the US. So, we could also write 15:00 as 3:00 pm est.

## Conclusion

In this project, we analyzed ask posts and show posts to determine which type of post and time receive the most comments on average. Based on our analysis, to maximize the amount of comments a post receives, we'd recommend the post be categorized as ask post and created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est).

However, it should be noted that the data set we analyzed excluded posts without any comments. Given that, it's more accurate to say that of the posts that received comments, ask posts received more comments on average and ask posts created between 15:00 and 16:00 (3:00 pm est - 4:00 pm est) received the most comments on average.